# Exploratory Data Analyis of MasteryConnect Student Test Data

![Student Data analysis](../notebook/images/Info5-980x604.png)

## Problem Statement
Teachers make on average, around 1500 decisions per day [link](https://www.continentalpress.com/blog/tips-for-reducing-teacher-decision-fatigue/#:~:text=According%20to%20data%20from%20busyteacher,just%20during%20the%20school%20day.) This may seem like a particularly dramatic statistic to some people outside the profession, but it is an enduring reality for those teachers who give their all in pursuit of positive academic outcomes. 
Tracking and monitoring student performance is crucial to acheiving this, however, manually doing so throughout the year, is not only challenging, but quite difficult to sustain. 
This project aims to alleviate this burden by:

- Collecting CSV assessment data from Mastery Connect
- Processing and uploading datasets to MongoDB
- Generating targeted insights using MongoDB's aggregation pipelines and pandas (a Python data manipulation library)
- Presenting these insights through intuitive visualizations

The key performance indicators generated by this system, such as standard mastery rates, question type proficiency, and individual student growth trajectories are invaluable insights that will empower teachers to implement data-driven classroom strategies, help students understand their progress, and allow administrators to identify broader trends.

Designed with scalability in mind, the system can potentially extend to multiple classrooms or entire school districts, significantly reducing decision fatigue for educators.

| Analysis Topic                    | Question                                                                 | Approach                                                                                                           |
|----------------------------------|--------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------|
| Overall Performance Analysis      | What is the overall distribution of student performance across all tests? | Analyze descriptive statistics of overall scores and visualize the distribution.                                   |
| Standard-Based Performance Analysis | How do students perform across different biology standards?               | a) Calculate and visualize average performance for each standard.<br>b) Identify top and bottom performing students for each standard.<br>c) Rank standards by difficulty based on average performance. |
| Question Type Analysis            | How does performance vary across different question types?               | Calculate and visualize difficulty rankings for different question types.                                           |
| Performance Analysis Over Time    | How has student performance changed over time?                           | Analyze and visualize performance trends over time for all students.                                                |
| Test Participation Analysis       | Are there patterns in test participation or missed tests?                 | Identify and analyze data for students who did not complete certain tests.                                          |
| Depth of Knowledge (DOK) Analysis | How does performance correlate with the depth of knowledge required?     | Analyze performance across different DOK levels.                                                                    |
| Class and Teacher Impact Analysis | Are there significant performance differences across classes or teachers? | Compare performance metrics across different classes and teachers.                                                  |
| Standard Coverage Analysis        | Which standards are most frequently tested, and how does this relate to performance? | Analyze the frequency of questions for each standard and correlate with performance data.                            |
| Individual Student Progress Tracking | How can we visualize and analyze individual student progress over time?    | Develop interactive visualizations for tracking individual student performance across tests and standards.        |
| Test Design Analysis              | Are there patterns in test composition that correlate with overall performance? | Analyze the relationship between test characteristics (e.g., question types, standards covered) and overall performance. |

### Imports

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from dotenv import load_dotenv
from IPython.display import display

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.components.db_connection import get_database, build_query 
from src.pipelines import mongo_db_pipelines as m_db
from src.components.data_ingestion import drop_collection


### Initialize database

In [ ]:
db = get_database(create_indices=True)
students = db['students']
tests = db['tests']

#### Filter data for particular teacher(s) or time period

In [ ]:
load_dotenv()
teacher = os.getenv('TEACHER_2')
class_name = ''
start_date = ''

query = build_query(teacher=teacher, class_name=class_name, start_date=start_date)

### Statistical analysis 
#### numeric data across all classes

In [ ]:
from src.utils import describe_numeric_field_pandas
# Average scores on tests
overall_scores_stats = describe_numeric_field_pandas(students, 'overall_percentage', query)


display(pd.DataFrame(overall_scores_stats))

#### Frequency distribution across all classes

This shows the distribution of fields (count values) by collection

In [ ]:
from src.utils import get_frequency_distribution

# MongoDB pipeline for frequency distribution
standard_dist = get_frequency_distribution(db=db, field='questions', subfield='standard', teacher=teacher)
item_type_dist = get_frequency_distribution(db=db, field='questions', subfield='item_type_name', teacher=teacher)
dok_dist = get_frequency_distribution(db=db, field='questions', subfield='dok', teacher=teacher)

# Renaming primary key 
standard_dist = pd.DataFrame(standard_dist).rename(columns={'_id':'standards'})
item_type_dist = pd.DataFrame(item_type_dist).rename(columns={'_id':'item_type'})
dok_dist = pd.DataFrame(dok_dist).rename(columns={'_id':'dok'})

print("Frequency of Course Standards")
display(pd.DataFrame(standard_dist).T)
print("Distribution of Question types")
display(pd.DataFrame(item_type_dist))
print("Depth of knowledge Frequency")
display(pd.DataFrame(dok_dist))

#### Distribution Visualizations


In [ ]:
from src.utils import create_frequency_distributions_plot
fig_distributions = create_frequency_distributions_plot(standard_dist, item_type_dist, dok_dist)
display(fig_distributions)

#### Performance over time for all students 

Aggreation of test results by date, not including students who missed a test (NaN values filtered out)

In [ ]:
from src.utils import performance_trend

# Run the performance trend analysis
performance_trend_df = pd.DataFrame(performance_trend(students, query))
print('First 5 tests by date and average')
display(performance_trend_df.head(5))

print('Last 5 tests by date and average')
display(performance_trend_df.tail(5))

#### Vizualization for performance trend overtime

In [ ]:
from src.utils import create_performance_trend_plot
fig_performance = create_performance_trend_plot(performance_trend_df)
display(fig_performance)

#### Students who did not complete a test 

In [ ]:
from src.utils import identify_students_with_nan, check_responses_for_nan

missed_test = identify_students_with_nan(students, 'overall_percentage')

# convert to data frame
missed_test_df = pd.DataFrame(missed_test)



### Anonymizing Data

In [ ]:
import random
from faker import Faker
fake = Faker()

# Custom function to generate fake test IDs
def fake_test_id():
    class_names = ['Biology', 'Chemistry', 'Anatomy']
    test_names = ['Midterm', 'Final', 'Quiz', 'Project', 'Exam']
    
    class_name = random.choice(class_names)
    test_name = random.choice(test_names)
    fake_date = fake.date_between(start_date='-1y', end_date='today').strftime('%Y%m%d')
    
    return f"{class_name}_{test_name}_{fake_date}"

# Create mappings for each column
def create_mapping(original_values, fake_function):
    return {value: fake_function() for value in set(original_values)}

student_id_mapping = create_mapping(missed_test_df['student_id'], lambda: f"S-{fake.numerify('######')}")
first_name_mapping = create_mapping(missed_test_df['first_name'], fake.first_name)
last_name_mapping = create_mapping(missed_test_df['last_name'], fake.last_name)
teacher_mapping = create_mapping(missed_test_df['teacher'], fake.name)
test_id_mapping = create_mapping(missed_test_df['test_id'], fake_test_id)

# Apply mappings to create anonymized dataframe
anonymized_df = missed_test_df.copy()
anonymized_df['student_id'] = anonymized_df['student_id'].map(student_id_mapping)
anonymized_df['first_name'] = anonymized_df['first_name'].map(first_name_mapping)
anonymized_df['last_name'] = anonymized_df['last_name'].map(last_name_mapping)
anonymized_df['teacher'] = anonymized_df['teacher'].map(teacher_mapping)
anonymized_df['test_id'] = anonymized_df['test_id'].map(test_id_mapping)

# Display the result
display(anonymized_df)


#### Addirional test stats

What tests did each student miss? 

In [ ]:
from src.utils import display_interactive_student_table

display_interactive_student_table(anonymized_df, width='80%', height= '400px')

### Overall Performance by Standard:
		Question: How has each student performed per standard? 

In [ ]:
# Execute the pipeline
pipeline = m_db.comprehensive_test_analysis_pipeline(query)
results = list(students.aggregate(pipeline))

# Convert to DataFrame
df = pd.DataFrame(results)


#### Anonymizing Dataframe to protect sensitive data. 

In [ ]:
import hashlib

def anonymize_df(df):
    # Create a copy of the dataframe
    anon_df = df.copy()
    
    # Create mappings for names and IDs
    student_id_map = {id: hashlib.md5(str(id).encode()).hexdigest()[:8] for id in df['student_id'].unique()}
    first_name_map = {name: fake.first_name() for name in df['first_name'].unique()}
    last_name_map = {name: fake.last_name() for name in df['last_name'].unique()}
    class_name_map = {class_name: f"Fake-class{fake.numerify('####')}" for class_name in df['class_name'].unique()}
    
    # Create a mapping for test_ids to ensure consistency
    test_id_map = {}
    for test_id in df['test_id'].unique():
        parts = test_id.split('_')
        if len(parts) >= 3:
            new_test_id = f"{class_name_map.get(parts[0], parts[0])}_{fake.word().capitalize()}_{fake.date_this_year().strftime('%Y%m%d')}"
        else:
            new_test_id = f"Test_{fake.word().capitalize()}_{fake.date_this_year().strftime('%Y%m%d')}"
        test_id_map[test_id] = new_test_id
    
    # Apply mappings
    anon_df['student_id'] = anon_df['student_id'].map(student_id_map)
    anon_df['first_name'] = anon_df['first_name'].map(first_name_map)
    anon_df['last_name'] = anon_df['last_name'].map(last_name_map)
    anon_df['class_name'] = anon_df['class_name'].map(class_name_map)
    anon_df['test_id'] = anon_df['test_id'].map(test_id_map)
    
    # Update test_name based on the new test_id
    anon_df['test_name'] = anon_df['test_id'].apply(lambda x: ' '.join(x.split('_')[1:-1]))
    
    
    return anon_df

# Apply the anonymization
anonymized_df2 = anonymize_df(df)

# Display the result
display(anonymized_df2.head(5))

In [ ]:
# General test analysis data (note: anonymized df is being used instead of df)
performance_by_standard = anonymized_df2.groupby(['student_id', 'first_name', 'last_name', 'standard', 'class_name']).agg({
    'is_correct': ['mean', 'count']
}).reset_index()
performance_by_standard.columns = ['student_id', 'first_name', 'last_name', 'standard', 'class_name', 'avg_performance', 'total_questions']
performance_by_standard['avg_performance'] *= 100  # Convert to percentage
performance_by_standard['avg_performance'] = performance_by_standard['avg_performance'].round(2) # Round off averages

display(performance_by_standard.head(5))

### Visualization of performance by standards for all students of all classes

In [ ]:
from src.utils import create_performance_dash_app
create_performance_dash_app(performance_by_standard)

### Overall Performance by Standard:
		Question: Who are the top performing students for each standard? 

In [ ]:
# Using Fake data (anonymized_df2 vs df)
def get_top_n_students(anonymized_df2, n=5, upper=True):
    # Create a rank within each standard
    anonymized_df2['rank'] = anonymized_df2.groupby('standard')['avg_performance'].rank(method='first', ascending=not upper)
    
    # Filter to keep only the top/bottom n ranks
    result = anonymized_df2[anonymized_df2['rank'] <= n]
    
    # Sort the final result to maintain the desired order
    result = result.sort_values(['standard', 'avg_performance', 'total_questions'], 
                                ascending=[True, not upper, False])
    
    # Drop the temporary rank column
    result = result.drop('rank', axis=1)
    
    return result.reset_index(drop=True)

# Get top 5 students for each standard
top_5_students = get_top_n_students(performance_by_standard, n=5, upper=True)
print("Top 5 Performing Students for Each Standard (first four shown):")
display(top_5_students.head(20))

### Overall Performance by Standard:
		Question: Who are the lowest performing students for each standard? 

In [ ]:
# Get lowest 5 students for each standard
bottom_5_students = get_top_n_students(performance_by_standard, n=5, upper=False)

print("Bottom 5 Performing Students for Each Standard (first four shown):")
display(bottom_5_students.head(20))

### Overall Performance by Standard:
		Question: Which standards are the most difficult as measured by average performance? 

#### Standards ranked by difficulty

In [ ]:
# Calculate average performance for each standard (using the anonymized df instead of df)
standard_difficulty = anonymized_df2.groupby('standard').agg({
    'is_correct': ['mean', 'count'],
    'student_id': 'nunique'
}).reset_index()

# Flatten column names
standard_difficulty.columns = ['standard', 'avg_performance', 'total_questions', 'unique_students']

# Convert average performance to percentage and round
standard_difficulty['avg_performance'] = (standard_difficulty['avg_performance'] * 100).round(2)

# Sort by average performance (ascending) to see most difficult standards first
standard_difficulty = standard_difficulty.sort_values('avg_performance', ascending=True)

# Add a difficulty rank
standard_difficulty['difficulty_rank'] = standard_difficulty['avg_performance'].rank(method='dense', ascending=True)

# Reset index again
standard_difficulty.reset_index(drop=True)

print("Standards Ranked by Difficulty (Most Difficult First):")
display(standard_difficulty)

### Overall Performance by Standard:
		Question: Which standards are students most proficient in?

In [ ]:
#Highest scoring standards 
display(standard_difficulty.sort_values(by='difficulty_rank', ascending=False))

### Standard performance overview

In [ ]:
from src.utils import display_standards_difficulty_graph

# Showing standards for chemistry as an example 
chm_stnds_diff = standard_difficulty[standard_difficulty['standard'].str.contains('Chm')]

display_standards_difficulty_graph(chm_stnds_diff)

### Question Type Difficulty
            Question: What are the difficulty rankings for different question types?
           
[What item types are in Mastery Connect?](https://community.canvaslms.com/t5/Mastery-Connect-Item-Authoring/What-item-types-can-I-create-in-Mastery-Connect/ta-p/562964)


In [ ]:
# Calculate difficulty for each question type
question_type_difficulty = anonymized_df2.groupby('question_type').agg({
    'is_correct': ['mean', 'count'],
    'student_id': 'nunique'
}).reset_index()

# Flatten column names
question_type_difficulty.columns = ['question_type', 'avg_performance', 'total_questions', 'unique_students']

# Convert average performance to percentage and round
question_type_difficulty['avg_performance'] = (question_type_difficulty['avg_performance'] * 100).round(2)

# Sort by average performance (ascending) to see most difficult question types first
question_type_difficulty = question_type_difficulty.sort_values('avg_performance', ascending=True)

# Add a difficulty rank
question_type_difficulty['difficulty_rank'] = question_type_difficulty['avg_performance'].rank(method='dense', ascending=True)

print("Question Types Ranked by Difficulty (Most Difficult First):")
display(question_type_difficulty)


#### Question type difficulty visualization

In [ ]:
from src.utils import display_question_type_difficulty_graph

display_question_type_difficulty_graph(question_type_difficulty)

### Student Progress Overtime:
		Question: How has students progress changed over time 

In [ ]:
# Using anonymized df
from src.utils import display_performance_figure
# Check if we have a date column
if 'date' not in anonymized_df2.columns:
    # Extract date from test_id if necessary
    anonymized_df2['date'] = pd.to_datetime(anonymized_df2['test_id'].str.split('_').str[-1], format='%Y%m%d')

# Ensure date is in datetime format
anonymized_df2['date'] = pd.to_datetime(anonymized_df2['date'])

display_performance_figure(anonymized_df2, performance_trend_df)